#### Imports

In [2]:
import sys
import os

import geopandas as gpd
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#### Constants

In [3]:
# General
#BASE_PATH = os.getcwd()
BASE_PATH = os.path.dirname(os.getcwd())

# Mapping
CHANGE_TYPE_MAP = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}
CHANGE_STATUS_MAP = {'Greenland': 0, 'Land Cleared': 1, 'Excavation': 1, 'Materials Dumped': 3, 'Prior Construction': 3, 'Materials Introduced': 4, 'Construction Started': 5, 'Construction Midway': 6, 'Construction Done': 8, 'Operational': 10, None: None}

# Data
COLORS = ['red', 'green', 'blue']
METRICS = ['std', 'mean']

# Columns groups
COLUMNS_TO_DROP = ['index']
DATE_COLUMNS = ['date0', 'date1', 'date2', 'date3', 'date4']
CHANGE_STATUS_COLUMNS = ['change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3', 'change_status_date4']
CHANGE_STATUS_VALUE_COLUMNS = ['change_status_value_date0', 'change_status_value_date1', 'change_status_value_date2', 'change_status_value_date3', 'change_status_value_date4']

#### Reading data

In [4]:
## Read data
train_df = gpd.read_file(f'{BASE_PATH}/data/1000samples.geojson', index_col=0)
# test_df = gpd.read_file(f'{BASE_PATH}/data/test.geojson', index_col=0)

#### Treatments

In [5]:
## Simple Treatments
train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors='coerce'))

#train_df['change_type'].map(CHANGE_TYPE_MAP)
train_df[CHANGE_STATUS_VALUE_COLUMNS] = train_df[CHANGE_STATUS_COLUMNS].replace(CHANGE_STATUS_MAP)


/tmp/ipykernel_166138/2232672186.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df[CHANGE_STATUS_VALUE_COLUMNS] = train_df[CHANGE_STATUS_COLUMNS].replace(CHANGE_STATUS_MAP)


In [12]:
num_samples = train_df.shape[0]
coef = np.zeros((num_samples))
time_ctt = 1e9*60*90*24
ones = np.ones((5,1))
for i in range(num_samples):
    y = np.array(train_df[CHANGE_STATUS_VALUE_COLUMNS].iloc[i].astype(float))
    y_nan_mask = np.isnan(y)
    if y_nan_mask.all():
        continue
    
    x = np.array(train_df[DATE_COLUMNS].iloc[i].astype(int))[:,np.newaxis]/time_ctt
    x = np.hstack((ones,x))
    y = y[:,np.newaxis]
    
    W = np.linalg.inv(x[~y_nan_mask,:].T@x[~y_nan_mask,:])@x[~y_nan_mask,:].T@y[~y_nan_mask,:]
    coef[i] = W[1]
    #print(y, train_df["change_type"].iloc[i])
train_df["civilizating_rate"] = coef



In [6]:
#print(train_df.head())